In [ ]:
!pip install PyPDF2
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers


In [ ]:
from google.colab import files
from PyPDF2 import PdfReader

# Upload the PDF
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

In [ ]:
# Extract text from the PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:

# Split the text into smaller chunks
def split_into_chunks(text, max_chunk_size=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_chunk_size):
        chunks.append(" ".join(words[i:i + max_chunk_size]))
    return chunks

In [ ]:
pdf_text = extract_text_from_pdf(pdf_path)
chunks = split_into_chunks(pdf_text)
print(f"Extracted {len(chunks)} chunks from the PDF.")

In [ ]:
# Use a free Hugging Face model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
import transformers
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
import torch
  # Lightweight and free model for embeddings

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up the Hugging Face pipeline
pipeline = transformers.pipeline(
    "text-generation",  # Task
    model="EleutherAI/gpt-neo-1.3B",  # Free text-generation model
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)



Device set to use cpu


In [ ]:
!pip install langchain_community

In [ ]:
 # Wrap the pipeline for LangChain compatibility
import langchain
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature': 0})


In [ ]:
# Load the SentenceTransformer model
model = SentenceTransformer(model_name)

# Generate embeddings for the chunks
chunk_embeddings = [model.encode(chunk) for chunk in chunks]  # Encode each chunk individually

# Store chunks and embeddings
chunk_data = [{"text": chunk, "embedding": embedding.tolist()} for chunk, embedding in zip(chunks, chunk_embeddings)]


In [ ]:
import faiss
import numpy as np

# Initialize FAISS index
embedding_dim = len(chunk_embeddings[0])  # Embedding dimension
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(np.array(chunk_embeddings))

# Save FAISS index and chunks
faiss.write_index(index, "climate_index.faiss")
np.save("chunks.npy", chunk_data)

print("Vector database created and saved.")


In [ ]:
# Function to search the FAISS index
def search_index(query, top_k=1):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = [{"text": chunk_data[i]["text"], "distance": distances[0][j]} for j, i in enumerate(indices[0])]
    return results

In [ ]:
# Test the search functionality
query = "What are the climate projections for India?"
results = search_index(query)
for idx, result in enumerate(results):
    print(f"Result {idx+1} (Distance: {result['distance']}):\n{result['text']}\n")

In [ ]:
results[0]['text']


'economic stability, and social well-being. With increasing unpredictability in precipitation patterns, it becomes increasingly important to develop adaptation strategies to build resiliency towards their potential negative effects. 34 ` CLIMATE SCENARIOS FOR INDIA (2021-2040) 35 ` Changes in Temperature (2021-2040) Climate related disasters in natural and human systems are considered to be intensiﬁed under warmer temperature thresholds. These events intensify depending on the extent of global warming, geographic locations, levels of development and vulnerability; and also how adaptation and mitigation measures are taken by the local authorities and followed by the public. Therefore, the change in temperature is a key parameter to measure and understand climate vulnerability for any region. Three Major Points ● The average summer maximum temperature of India will have a 1.5 degree Celsius increase by 2043 according to the "Middle of the Road" emission scenario and by 2041 for the "Foss

In [ ]:
# from transformers import pipeline
# from huggingface_hub import notebook_login

# # Log in to Hugging Face
# notebook_login()

# # Now, load the language model
# generator = pipeline("text-generation", model="meta-llama/Meta-Llama-3.1-8B-Instruct")

In [ ]:

! huggingface-cli login

In [ ]:
# Generate prompt
def generate_prompt(results, query):
    # Check if the query is a greeting
    greetings = ["hi", "hello", "hey", "hai", "greetings"]
    if query.lower().strip() in greetings:
        return (
            "Hello! I am a climate Q&A chatbot powered by the report 'Navigating India's Climate Future' "
            "by Azim Premji University. I can answer your questions about India's climate projections, "
            "impacts, and strategies for adaptation and resilience based on the report. How can I assist you today?"
        )

    # Handle cases where the report context is empty
    if not results:
        return (
            "I'm sorry, I couldn't find any relevant information in the uploaded report. "
            "Please try rephrasing your query or ask a different question."
        )

    # Combine the retrieved results into context
    context = "\n".join([f"Chunk {i+1}: {result}" for i, result in enumerate(results)])

    # Define the main prompt for Q&A
    prompt = (
        f"You are a climate Q&A chatbot powered by the report 'Navigating India's Climate Future' "
        f"by Azim Premji University. Your role is to assist users by answering their questions "
        f"based on the information provided in the report. Always ensure your answers are accurate "
        f"and derived solely from the given context. If the context does not contain enough information, "
        f"respond by saying, 'I'm sorry, the report does not contain information about that.'\n\n"
        f"Report Context:\n{context}\n\n"
        f"User Question: {query}\n"
        f"Answer:"
    )
    return prompt





In [ ]:
def generate_response(query):
    results = search_index(query)
    prompt = generate_prompt(results, query)

    if "I couldn't find any relevant information" in prompt:
        return prompt  # Fallback message

    # Truncate the prompt if necessary
    max_tokens = 1024
    tokens = generator.tokenizer(prompt, truncation=True, max_length=max_tokens, return_tensors="pt")
    truncated_prompt = generator.tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)

    # Generate the response
    response = generator(
        truncated_prompt,
        max_new_tokens=150,
        num_return_sequences=1,
        pad_token_id=generator.tokenizer.eos_token_id
    )
    return response[0]['generated_text']


In [ ]:

# Test the chatbot
query = "How will climate change affect agriculture in India?"
response = generate_response(query)
print(f"Chatbot: {response}")

Chatbot: You are a climate Q&A chatbot powered by the report 'Navigating India's Climate Future' by Azim Premji University. Your role is to assist users by answering their questions based on the information provided in the report. Always ensure your answers are accurate and derived solely from the given context. If the context does not contain enough information, respond by saying, 'I'm sorry, the report does not contain information about that.'

Report Context:
Chunk 1: {'text': "spread to central India. ● A drier winter in the northwestern Indian region may feel semi-arid due to rising temperatures and less water availability. 50 ` ● The irrigation system is the backbone of rabi crop productivity. As a result, lack of water may result in a signiﬁcant reduction in rabi crop productivity, causing farmers to suffer heavy losses. As a result, people may look for new occupations and migrate across. ● Mass migration may occur to more habitable and agriculturally sustainable regions of east

In [ ]:
response

'You are a climate Q&A chatbot powered by the report \'Navigating India\'s Climate Future\' by Azim Premji University. Your role is to assist users by answering their questions based on the information provided in the report. Always ensure your answers are accurate and derived solely from the given context. If the context does not contain enough information, respond by saying, \'I\'m sorry, the report does not contain information about that.\'\n\nReport Context:\nChunk 1: {\'text\': "spread to central India. ● A drier winter in the northwestern Indian region may feel semi-arid due to rising temperatures and less water availability. 50 ` ● The irrigation system is the backbone of rabi crop productivity. As a result, lack of water may result in a signiﬁcant reduction in rabi crop productivity, causing farmers to suffer heavy losses. As a result, people may look for new occupations and migrate across. ● Mass migration may occur to more habitable and agriculturally sustainable regions of e

In [ ]:
print("Climate Chatbot: Ask me questions based on 'Navigating India's Climate Future'. Type 'exit' to quit.")
while True:
    query = input("You: ")
    if query.lower() in ['exit', 'quit']:
        print("Climate Chatbot: Goodbye!")
        break
    response = generate_response(query)
    print(f"Climate Chatbot: {response}")


Climate Chatbot: Ask me questions based on 'Navigating India's Climate Future'. Type 'exit' to quit.
You: hai
Climate Chatbot: Hello! I am a climate Q&A chatbot powered by the report 'Navigating India's Climate Future' by Azim Premji University. I can answer your questions about India's climate projections, impacts, and strategies for adaptation and resilience based on the report. How can I assist you today?
You: how heatwaves affect climate change in Kerala?
Climate Chatbot: You are a climate Q&A chatbot powered by the report 'Navigating India's Climate Future' by Azim Premji University. Your role is to assist users by answering their questions based on the information provided in the report. Always ensure your answers are accurate and derived solely from the given context. If the context does not contain enough information, respond by saying, 'I'm sorry, the report does not contain information about that.'

Report Context:
Chunk 1: {'text': 'economic stability, and social well-being.

In [ ]:
print("Climate Chatbot: Ask me questions based on 'Navigating India's Climate Future'. Type 'exit' to quit.")
while True:
    query = input("You: ")
    if query.lower() in ['exit', 'quit']:
        print("Climate Chatbot: Goodbye!")
        break
    response = generate_response(query)
    print(f"Climate Chatbot: {response}")


Climate Chatbot: Ask me questions based on 'Navigating India's Climate Future'. Type 'exit' to quit.
Climate Chatbot: Hello! I am a climate Q&A chatbot powered by the report 'Navigating India's Climate Future' by Azim Premji University. I can answer your questions about India's climate projections, impacts, and strategies for adaptation and resilience based on the report. How can I assist you today?

We use cookies to understand how you use our site and to improve user experience. This includes personalising content and advertising. By continuing to use our site, you accept our use of cookies, revised Privacy Policy. Read more.

India's Climate Projections and Impacts

India

Catherine A. Cawood

November 20, 2009 11:14 AM

India's climate projections and impacts

Climate projections and impacts

Catherine A. Cawood is Associate Professor in the Department of Geological Sciences at the University of Michigan. She has more than 40 years of scientific and technical experience in Earth sy

In [ ]:
faiss.write_index(index, "climate_index.faiss")
np.save("chunks.npy", chunk_data)


In [ ]:
index = faiss.read_index("climate_index.faiss")
chunk_data = np.load("chunks.npy", allow_pickle=True)


In [ ]:
# Install Gradio
!pip install gradio

In [ ]:
import gradio as gr

# Function to interact with the chatbot
def chatbot_interface(query):
    response = generate_response(query)
    return response

# Create Gradio interface
interface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(label="Enter your question about India's climate future:"),
    outputs=gr.Textbox(label="Chatbot Response",interactive=True),
    title="Climate Chatbot",
    description="Ask questions based on the report 'Navigating India's Climate Future'.",
)

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d56cbc05174f3fa48.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
with gr.Blocks() as demo:
  inputs=gr.Textbox(label="Enter your question about India's climate future:")
  outputs=gr.Textbox(label="Chatbot Response",interactive=True)
  submit=gr.Button('submit')
  #submit.click(generate_response,inputs,outputs,queue=False)
  @submit.click(inputs=inputs,outputs=outputs)
  def submit(inputs):
    return generate_response(inputs)
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa012f56b135fa905b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
response

NameError: name 'response' is not defined